<a href="https://colab.research.google.com/github/atifR/Waste_Classification/blob/main/predict_resNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AI model to predict waste images.
The dataset is acquired from
https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification/data


In [17]:

import os
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torchvision.transforms.v2 as v2
from torchvision import transforms
import torch
import torchvision
from torch import nn, optim
from torchsummary import summary
import time

In [2]:
root_folder = ""

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## unzip to colab temo for faster access!

In [4]:
!unzip -q -d "/content/Waste_dataset" "/content/drive/MyDrive/AI Projects/Waste_Classification/Dataset.zip"

In [5]:
root_folder = "/content/Waste_dataset"

### Parameters

In [6]:
dataset_folder  = root_folder+ "/archive/images/images"
batch_size = 32
train_ratio = 0.7
valid_ratio = 0.15
test_ratio = 0.15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

print (f"Settings ... \n dataset path \t\t {dataset_folder} \n batch_size \t\t {batch_size}" +
       f"\n train_ratio \t\t {train_ratio} \n validation ratio \t {valid_ratio} \n test ratio \t\t {test_ratio} \n device \t\t {device}")

Using device: cuda
Settings ... 
 dataset path 		 /content/Waste_dataset/archive/images/images 
 batch_size 		 32
 train_ratio 		 0.7 
 validation ratio 	 0.15 
 test ratio 		 0.15 
 device 		 cuda


In [7]:
class_labels = [name for name in os.listdir(dataset_folder) if not name.startswith(".DS_Store")]
print(class_labels)
num_classes = len(class_labels)
print(num_classes)

['newspaper', 'magazines', 'plastic_food_containers', 'food_waste', 'plastic_trash_bags', 'aerosol_cans', 'disposable_plastic_cutlery', 'aluminum_soda_cans', 'glass_food_jars', 'coffee_grounds', 'paper_cups', 'plastic_shopping_bags', 'cardboard_packaging', 'shoes', 'steel_food_cans', 'tea_bags', 'plastic_cup_lids', 'styrofoam_food_containers', 'plastic_straws', 'clothing', 'styrofoam_cups', 'cardboard_boxes', 'office_paper', 'aluminum_food_cans', 'plastic_detergent_bottles', 'plastic_water_bottles', 'eggshells', 'glass_beverage_bottles', 'plastic_soda_bottles', 'glass_cosmetic_containers']
30


In [9]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# transformations = v2.Compose([
#     v2.ToImage(),
#     v2.RandomHorizontalFlip(),
#     v2.RandomVerticalFlip(),
#     v2.RandomResizedCrop(size=(224, 224), antialias=True),
#     v2.RandomRotation(degrees = (0, 170)),
#     v2.ToDtype(dtype = torch.float32, scale = True),
#     v2.Resize(size = (224, 224), antialias = True),
#     v2.ToTensor(),
#     v2.Normalize(mean, std)
# ])

transformations = transforms.Compose([
                    #transforms.Resize((224,224)),
                    transforms.ToTensor(),
                    #transforms.Normalize(mean,std)

])


dataset = ImageFolder(dataset_folder,transform=transformations)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


### Creating and splitting the dataset

In [10]:
train_size = int(len(dataset) * train_ratio)
valid_size = int(len(dataset) * valid_ratio)
test_szie = int(len(dataset) * test_ratio)

train_dataset, valid_dataset, test_dataset = random_split(dataset,[train_size, valid_size, test_szie])

train_dataloader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset,shuffle=True,batch_size=batch_size)
test_dataloader = DataLoader(test_dataset,shuffle=True, batch_size=batch_size)

print(f"Lenght of training dataset {len(train_dataloader.dataset)}")
print(f"Lenght of validation dataset {len(valid_dataloader.dataset)}")
print(f"Lenght of test dataset {len(test_dataloader.dataset)}")


Lenght of training dataset 10500
Lenght of validation dataset 2250
Lenght of test dataset 2250


In [13]:
import time
start = time.time()
x,y = next(iter(train_dataloader))
print(x.shape)
print(y.shape)

end = time.time()
print(f" time {end-start} secs")

torch.Size([32, 3, 256, 256])
torch.Size([32])
 time 0.18160343170166016 secs


## CNN model

In [ ]:
class CNN(nn.Module):
    def __init__ (self):
        super().__init__()
        self.network = nn.Sequential(

            nn.Conv2d(in_channels = 3, out_channels = 4, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
                                                                                                    #256,256,4
            nn.Conv2d(in_channels = 4, out_channels = 8, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
                                                                                                    #256,256,8
            nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #128,128,16
            nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #64,64,32
            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #32,32,64
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #16,16,64
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
                                                                                                    #8,8,64
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
                                                                                                    #8,8,128
            nn.Flatten(),
            nn.Linear(128*8*8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),

        )


    def forward(self, x):
            return self.network(x)

### ResNet

In [11]:
res_net = torchvision.models.resnet50(weights='IMAGENET1K_V1')
#res_net.parameters().requires_grad = False
res_net.fc  = torch.nn.Sequential(
    nn.Linear(2048,128),
    nn.ReLU(),
    nn.Linear(
        in_features=128,
        out_features=num_classes
    ),
)

res_net.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 183MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model = CNN()
model = model.to(device)    #converting model to gpu from cpu
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3 )
print("Compiled Successfully")

Compiled Successfully


In [ ]:
from torchsummary import summary
model = res_net
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(res_net.fc.parameters())

In [13]:
def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x,y = x.to(device), y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            correct += (predictions == y).sum().item()
            total += predictions.size(0)
    model.train()
    return correct/total

In [15]:


def train_model(model,criterion,optimier, data_loader,device, num_epochs=0):
    model.to(device)
    for epoch in range(num_epochs):
        #loss_batches = 0
        loss_epoch=0;
        corrects_batches = 0
        count = 0
        start = time.time()
        correct = 0
        total = 0
        for x,y in data_loader:

            x,y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs,y)
            optimier.zero_grad()
            loss.backward()
            optimier.step()
            _,preds = torch.max(outputs,1)
            correct += (preds == y).sum().item()
            total += preds.size(0)
            loss_epoch += loss.item()
            count += 1
            #loss_batches += loss.item() * x.size(0)
            #corrects_batches += torch.sum(preds == y.data)
        #epoch_loss = loss_batches / len(data_loader)
        epoch_acc = correct / total
        print(f"\n epoch {epoch} Loss : {(loss_epoch/count):.4f} Accuracy {epoch_acc:.2f}, time : {time.time()-start} secs")
        if (epoch % 3 == 0 ):
          torch.save(res_net.state_dict(),f"resNet_{epoch:02d}_{epoch_acc:.2f}.h5")
    return model




In [18]:
train_model(res_net,criterion=criterion, optimier=optimizer,
            data_loader=train_dataloader, device=device, num_epochs=10)

torch.save(res_net.state_dict(),'resNet.h5')



 epoch 0 Loss : 2.4113 Accuracy 0.34, time : 169.0288965702057 secs

 epoch 1 Loss : 1.7117 Accuracy 0.49, time : 167.25494480133057 secs

 epoch 2 Loss : 1.5481 Accuracy 0.54, time : 166.7708089351654 secs

 epoch 3 Loss : 1.4808 Accuracy 0.55, time : 167.54358506202698 secs

 epoch 4 Loss : 1.4145 Accuracy 0.57, time : 167.262140750885 secs

 epoch 5 Loss : 1.3929 Accuracy 0.58, time : 167.32599782943726 secs

 epoch 6 Loss : 1.3823 Accuracy 0.59, time : 166.9162313938141 secs

 epoch 7 Loss : 1.3567 Accuracy 0.58, time : 168.16985249519348 secs

 epoch 8 Loss : 1.3453 Accuracy 0.59, time : 166.92668890953064 secs

 epoch 9 Loss : 1.3215 Accuracy 0.59, time : 167.76110410690308 secs


In [ ]:

print(f"Training Accuracy is {calculate_accuracy(train_dataloader, model)*100}")

print(f"Testing Accuracy is {calculate_accuracy(test_dataloader, model)*100}")